## Berechnungen von Merkmalen

In diesem Notebook ist es möglich den Mittelwert, Varianz und Standardabweichung für die aufgenommenen Messwerte aus der Datei "merkmaleRoh.csv" der Android TM Fahrradwege Monitoring App zu berechnen. Zu einem Bild werden parallel Messwerte erfasst von dem Beschleunigungssensor und Magnetsensor des Smartphones, innerhalb eines Zeitfenster mit einer Länge von 80ms (siehe hierzu auch das Notebook "gatheringSequence" in dem Ordner datenvisualisierung). Bei dem Beschleunigungssensor sind dies die Messwerte der beschleunigung von den 3-Achsen X, Y sowie Z (Abb. 1). Des Weiteren werden die Gier-Nick-Roll Winkel in rad in einem Zeitfenster erfasst. Für die Berechnung des Mittelwerts, Varianz und Standardabweichung über die Messwerte wird die Form der abgespeicherten Messwerte geändert, weil diese als Python Objekte verwaltet werden mit welchen nicht gerechnet werden kann. Zu den Messwertreihen wird jeweils eine CSV Datei erstellt. Eine Reihe stellt dann in dieser Datei eine Messreihe zu einem Zeitfenster dar. Bei der ausführung dieses Notebooks werden deshalb zusätzlich folgende CSV Dateien erstellt:
   
Y-Achse Beschleunigungswerte in m/s^2  
Z-Achse Beschleunigungswerte in m/s^2    
Nick Messwerte in rad    
Zeitstempel der Messwerte in ns  

<img src="../daten/abbildungen/sensorOrientierung.png" alt="Android Smartphone Sensor Orientierung" width="200px" />
Abbildung 1: 3-Achsen Smartphone Sensor-Koordinatensystem   
Quelle: siehe Bildverzeichnis Abb. 2 in der README.md Datei

In [54]:
import pandas
import numpy as np
import numpy.testing as npt
import math
import os

In [102]:
dataset = '51'

In [103]:
featuresDf = pandas.read_csv("../daten/merkmale_datensatz_" + dataset + "/merkmaleRoh.csv")
featuresDf.describe()

,Unnamed: 0,Zeitstempel in Unixzeit,Breitengrad,Laengengrad,Geschwindigkeit in km/h,Genauigkeit der Geschwindigkeit speedAccuracyMetersPerSecond,Lokations Zeitstempel in Unixzeit,Anzahl der Messwerte,Start des Zeitfensters in ns seit Start der JVM,Zeitstempel Messwertdaten anfordern in Unixzeit,Start der Messwerterfassung in ns seit Start der JVM,Erster Zeitstempel der Teilliste in ns seit Start der JVM,Start der Belichtung in ns seit Start der JVM,Belichtungszeit in ns,Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM,Speicherzeitpunkt der Merkmale in Unixzeit
count,9454.000000,9.454000e+03,9454.000000,9454.000000,9454.000000,9454.000000,9.454000e+03,9454.000000,9.454000e+03,9.454000e+03,9.454000e+03,9.454000e+03,9.454000e+03,9.454000e+03,9.454000e+03,9.454000e+03
mean,4726.500000,1.530620e+12,53.617201,10.191359,15.348372,0.760315,1.530620e+12,29.554474,2.219282e+13,1.530620e+12,2.216347e+13,2.219226e+13,2.219294e+13,5.876306e+06,2.219307e+13,1.530620e+12
std,2729.279056,8.904760e+05,0.004117,0.016362,3.244137,0.439665,8.904775e+05,8.698953,8.904868e+11,8.904673e+05,8.980231e+11,8.905077e+11,8.904778e+11,7.642593e+06,8.904675e+11,8.904655e+05
min,0.000000,1.530619e+12,53.610435,10.161959,5.004000,0.092195,1.530619e+12,14.000000,2.054263e+13,1.530619e+12,2.054186e+13,2.054186e+13,2.054292e+13,5.883430e+05,2.054324e+13,1.530619e+12
25%,2363.250000,1.530619e+12,53.613921,10.179405,13.284001,0.464676,1.530619e+12,24.000000,2.149235e+13,1.530619e+12,2.144163e+13,2.149186e+13,2.149252e+13,1.746644e+06,2.149268e+13,1.530619e+12
50%,4726.500000,1.530620e+12,53.616451,10.189540,15.660000,0.660681,1.530620e+12,27.000000,2.219535e+13,1.530620e+12,2.218063e+13,2.219484e+13,2.219555e+13,2.822210e+06,2.219575e+13,1.530620e+12
75%,7089.750000,1.530621e+12,53.620819,10.204003,17.568003,0.934773,1.530621e+12,32.000000,2.296505e+13,1.530621e+12,2.295950e+13,2.296456e+13,2.296516e+13,5.809891e+06,2.296530e+13,1.530621e+12
max,9453.000000,1.530622e+12,53.625130,10.222290,28.404000,3.715279,1.530622e+12,89.000000,2.367700e+13,1.530622e+12,2.365682e+13,2.367646e+13,2.367735e+13,2.999632e+07,2.367771e+13,1.530622e+12


In [92]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8708 entries, 0 to 8707
Data columns (total 20 columns):
Unnamed: 0                                                            8708 non-null int64
Zeitstempel in Unixzeit                                               8708 non-null int64
Breitengrad                                                           8708 non-null float64
Laengengrad                                                           8708 non-null float64
Geschwindigkeit in km/h                                               8708 non-null float64
Genauigkeit der Geschwindigkeit speedAccuracyMetersPerSecond          8708 non-null float64
Lokations Zeitstempel in Unixzeit                                     8708 non-null int64
Z-Achse Beschleunigungswerte in m/s^2                                 8708 non-null object
Y-Achse Beschleunigungswerte in m/s^2                                 8708 non-null object
Nick Messwerte in rad                                                 870

Die folgenden Spalten des Datenframes haben als Type ein Python object.   
Y-Achse Beschleunigungswerte in m/s^2   
Z-Achse Beschleunigungswerte in m/s^2      
Nick Messwerte in rad                           
Zeitstempel der Messwerte in ns               
Implizit haben diese Spalten als Typ Strings. Aber in einem DataFrame werden Strings als    
Python objects erkannt. Um den Datentyp auf float zu casten wird aus den Spalten zunächst ein Liste erstellt.    
Welche dann wiederum als Numpy Array auf den Typ float konvertiert wird.   

In [104]:
accYList = featuresDf['Y-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(accYList[0]).astype(float)).transpose())
for x in accYList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accYDf = pandas.concat(accList, ignore_index=True)

In [105]:
# Konvertiere Beschleunigungssensordaten Z-Achse zu einem DataFrame
measuredDataList = featuresDf['Z-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accZDf = pandas.concat(accList, ignore_index=True)

In [106]:
# Konvertiere Nick Messwerte zu DataFrame
measuredDataList = featuresDf['Nick Messwerte in rad'].str.split(" ").tolist()
pitchList = []
pitchList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    pitchList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
nickDf = pandas.concat(pitchList, ignore_index=True)

In [107]:
measuredDataList = featuresDf['Zeitstempel der Messwerte in ns'].str.split(" ").tolist()
timestampsList = []
timestampsList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    timestampsList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
sensorTimestampsDf = pandas.concat(timestampsList, ignore_index=True)

In [108]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'] = accYDf.T.mean()
featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'] = accZDf.T.mean()
featuresDf['Mittelwert Nick in rad'] = nickDf.T.mean()

Berechnet die Varianz. Dieser Funktion muss als Paramter der Mittelwert (mean) und der DataFrame mit   
den Float Werten übergeben werden, um die Varianz zu berechnen. Als Varianz wird der Durchschnitt der quadrierten  
Differenzen zum Mittelwert bezeichnet.   
Prec.:    
Postc.: Gibt die berechnete Variance als Float zurück oder 0 wenn die Anzahl der Werte <= 0  

In [23]:
def calcVariance(meansDf, dfValues):
    variance = []
    for i, values in dfValues.iterrows():
        sum = 0
        for value in values:
            if math.isnan(value) == False:
                tempDifference = value-meansDf[i]
                sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / values.count(),5)))
    return np.array(variance).astype(float)

Hier wird die Standardabweichung der Varianz berechnet. Dies ist die Wurzel der Varianz.   
Für die Berechnung wird der Absolutwert von der Varianz genommen.    
Prec.:
Postc.:  Standardabweichung wird zurückgegeben.

In [24]:
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        temp = math.sqrt(np.abs(v))
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

Die Funktion berechnet die Winkeldifferenz in rad zwischen den ersten gemessenen Gierwinkel und den letzten Gierwinkel   
innerhalb einer als Argument übergebenen DataFrames. Der zurückgegebene Radiant    
ist immer positiv und gibt die relative änderung in rad an.      
Prec.:   
Postc.: Radiant berechnet   

In [25]:
def calculateAngelChangeAzimuth(azimuthDf):
    result = []
    for i,values in azimuthDf.iterrows():
        first = values[0]*(180/math.pi)
        last = values.iloc[values.count()-1]*(180/math.pi)
        resultTemp = np.abs(first - last)
        if(resultTemp > 180):
            resultTemp = 360 - resultTemp
        result.append("{0:.5f}".format(round(resultTemp * ( math.pi / 180),5)))
    return np.array(result).astype(float)

In [109]:
# Berechnung der Varianzen und hinzufügen der Ergebnisse als neue Spalte in dem DataFrame 
featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'],accYDf)
featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'],accZDf)
featuresDf['Varianz Nick in rad'] = calcVariance(featuresDf['Mittelwert Nick in rad'],nickDf)

In [110]:
# Berechne Standardabweichung
featuresDf['Standardabweichung Y-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Nick in rad'] = calculateStandardDeviation(featuresDf['Varianz Nick in rad'])

In [111]:
# Änderungen in CSV Datei schreiben
featuresDf.set_index('Zeitstempel in Unixzeit', inplace=True)
featuresDf.to_csv('../daten/merkmale_datensatz_'+ dataset + '/merkmale.csv')

In [52]:
featuresDf = pandas.read_csv("../daten/merkmale_datensatz_" + dataset + "/merkmale.csv")

In [112]:
# Speichert die DatenFrames mit den Messwerten in csv Dateien
accYDf.index.names = ["index"]
accYDf.to_csv('../daten/merkmale_datensatz_' + dataset + '/yAchseBeschleunigungswerte.csv')
accZDf.index.names = ["index"]
accZDf.to_csv('../daten/merkmale_datensatz_' + dataset + '/zAchseBeschleunigungswerte.csv')
nickDf.index.names = ["index"]
nickDf.to_csv('../daten/merkmale_datensatz_' + dataset + '/nickMesswerte.csv')
sensorTimestampsDf.index.names = ["index"]
sensorTimestampsDf.to_csv('../daten/merkmale_datensatz_' + dataset + '/sensoreventZeitstempel.csv')

In [ ]:
# Unittests
import unittest

class CalcTest(unittest.TestCase):
    
    
    # Testet die Funktion calcVariance mit einem DataFrame mit den Werten [2.0, 2.0]
    # Der Mittelwert sollte 2.0 sein
    # Das erwartete Ergebnis ist 0.0
    def testCalcVariance2(self):
        meanL = [2.0]
        meanDf = pandas.DataFrame(meanL)
        valuesA = [2.0,2.0]
        valuesDf = pandas.DataFrame([[2.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 0.0,2)
        
    # Testet die Funktion calcVariance mit einem DataFrame mit den folgenden Werten:
    # [1.24, 2.5213, 10.434, 42.45, 5.9]
    # Der Mittelwert ist 12.508
    # Die erwartete Varianz ist 234,247016
    def testCalcVariance5(self):
        meanDf = pandas.DataFrame([12.508])
        valuesDf = pandas.DataFrame([[1.24, 2.52, 10.43, 42.45, 5.9]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 234.24702,5)   
    
    # Testet calcVariance mit 0 Datenwerten   
    def testCalcVariance0(self):
        meanDf = pandas.DataFrame([0.0])
        valuesDf = pandas.DataFrame([[0.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),0.0,2)  
        
    # Testet calcVariance mit negativen Datenwerten   
    def testCalcVarianceNegative(self):
        meanDf = pandas.DataFrame([-24.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),451.5,1) 
        
    # Testet calcVariance mit Muultidimensionalen DataFrame   
    def testCalcVarianceMultidim(self):
        meanDf = pandas.DataFrame([-24.0,0.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0],[0.0,0.0,0.0,0.0]])
        # Hier wird Numpy Test verwendet um das Ergebnis (zwei Arrays) zu vergleichen
        npt.assert_array_equal(calcVariance(meanDf[0],valuesDf),np.array([ 451.5,0.0])) 
        
    # Dieser Test testet die Funktion calculateStandardDeviation.
    def testCalculateStandardDeviation(self):
        varianceDf = pandas.DataFrame([451.5])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),21.24853,5) 
        
    # Testet die Funktion calculateStandardDeviation.
    # Dabei hat das Argument den Wert 0.0.
    def testCalculateStandardDeviationZero(self):
        varianceDf = pandas.DataFrame([0.0])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),0.0,2)
    
    #  Testet die Funktion calculateAngelChangeAzimuth mit zwei gleichen rad Werten
    def testCalculateAngelChangeAzimuthEqual(self):
        radiants = pandas.DataFrame([[1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),0.0,2)
    
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten -1.0 und 1.0
    def testCalculateAngelChangeAzimuthOne(self):
        radiants = pandas.DataFrame([[-1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 1.0 und -1.0
    def testCalculateAngelChangeAzimuthOne2(self):
        radiants = pandas.DataFrame([[1.0,-1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  

    # Testet die Funktion calculateAngelChangeAzimuthValues mit mehreren Werten
    def testCalculateAngelChangeAzimuthValues(self):
        radiants = pandas.DataFrame([[-1.57,1.0,2.0,1.3,1.57]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),3.14,2)
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 0.05 und 0.06
    def testCalculateAngelChangeAzimuthSmall(self):
        radiants = pandas.DataFrame([[0.01,0.02]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 0.01,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde mit 
    # negative Radianten 
    def testCalculateAngelChangeAzimuthNegativ(self):
        radiants = pandas.DataFrame([[-2.9,-2.0,-1.28]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 1.62,2) 

unittest.main(argv=[''], verbosity=2, exit=False)